# Merging CSV files
Code used to merge CSV files

In [52]:
# Import necessary libraries.
import numpy as np
import pandas as pd
from sklearn import preprocessing
print("Setup Complete")

Setup Complete


In [53]:
# Import the data.

composition_data = pd.read_csv("/content/ATP3-UFS-Composition.csv", index_col=0, parse_dates=['DATETIME'])
input_data = pd.read_csv("/content/ATP3-UFS-SummaryCombinedData.csv", index_col=0, parse_dates=['DateTime'])
instrumentation_data = pd.read_csv("/content/ATP3-UFS-Instrumentation-daily.csv", index_col = 0, parse_dates=['Date'])
weather_data = pd.read_csv("/content/ATP3-UFS-Weather-daily.csv", index_col = 0, parse_dates=['Date'])

In [54]:
#Clean the Data

input_data= input_data[input_data['Analytical.Sample.ID'].notna()]
composition_data = composition_data.set_index("Analytical.Sample.ID")
input_data = input_data.set_index("Analytical.Sample.ID")
combined_data = composition_data.merge(input_data)
pd.DataFrame.drop_duplicates(combined_data)
combined_data = combined_data.drop(['NH4.mg.L', 'NH4.PCT.RSD', 'crash', 'Comments', 'OD680', 'OD680.PCT.RSD',
                                         'NO3.supplied.g', 'NO3.utilized.g', 'NO3.efficiency..', 'NH4.supplied.g',	'NH4.utilized.g',
                                         'NH4.efficiency..', 'P.supplied.g',	'P.utilized.g',	'P.efficiency..', 'mg.AFDW_mol.PAR.input',
                                         'mg.AFDW_kW.GLE.input'], axis=1).rename(columns={'DATETIME': 'Date'})

In [55]:
#Prepare the Data for Merging

combined_data['Date'] = pd.to_datetime(combined_data['Date'], format='%d%b%Y')
combined_data['Date'] = combined_data['Date'].dt.date
instrumentation_data['Date'] = pd.to_datetime(instrumentation_data['Date'], format='%d%b%Y')
weather_data['Date'] = pd.to_datetime(weather_data['Date'], format='%d%b%Y')
combined_data['SiteID2'] = combined_data['SiteID']
combined_data = combined_data.set_index(['SiteID', 'PondID', 'Date'])
instrumentation_data = instrumentation_data.set_index(['SiteID', 'Date'])

Merge the Data


In [56]:
mid_merge = instrumentation_data.merge(weather_data, left_index=True, right_on=['SiteID', 'Date'])
combined_data = combined_data.merge(mid_merge, left_index=True, right_on=['SiteID', 'PondID', 'Date'])

In [57]:
#export and test the csv file
combined_data.to_csv("/content/Combined Data.csv")
combined_data

,Protein.AF,FAME.Lipids.AF,Carbohydrates.AF,ExperimentID,StrainID,SourceID,BatchID,Duration.days,DateTime,TreatmentID,time.between.harvests.days,Harvest.,Harvest.Vol..L.,AFDW..g.L.,AFDW..g.,Depth.cm,Evaporation.Rate.cm.day,pH_x,Salinity,Temp.C,NO3.mg.L,NO3.PCT.RSD,P.mg.L,P.PCT.RSD,N.P.ratio,Tracking.ID,DW.g.L,DW.RSD,AFDW.g.L,AFDW.RSD,Ash,OD750,OD750.PCT.RSD,NO3.demand.g.N.g.AFDW,NH4.demand.g.N.g.AFDW,P.demand.g.P.g.AFDW,SiteID2,PondID,StrainID_x,pH_y,Temp.avg (C),Temp.max (C),Temp.min (C),Cond (mS.cm),DO (mg.L),DO (%sat),Sal (g.L),PAR (umol.m2.s),SiteID,StrainID_y,Date,AirTemp(C),RH(%),GlobalLightEnergy(W.m2),WindSpeed(km.hr),WDir(deg),Precip.tot(cm)
ExperimentID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
OCT172013,43.86,10.77,6.26,OCT172013,KA32,KA32_10042013_CRE,KA32_10172013_SPW,6.73,2013-10-24,NaN,NaN,NaN,NaN,NaN,NaN,24.13,0.96,8.34,NaN,12.2,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.59,0.12,NaN,NaN,NaN,ASU,P1,KA32,8.47,17.02,21.9,11.9,52297.48,7.01,89.55,34.46,0.0,ASU,KA32,2013-10-24,22.79,24.90,186.99,4.56,198.29,0.00
OCT172013,44.32,NaN,8.45,OCT172013,KA32,KA32_10042013_CRE,KA32_10172013_SPW,12.74,2013-10-30,NaN,NaN,NaN,NaN,NaN,NaN,24.13,0.46,8.71,NaN,9.6,23.60,3.06,1.06,0.66,49.07,3.0,0.34,1.48,0.33,0.59,5.47,1.40,0.55,0.151,NaN,NaN,ASU,P1,KA32,8.57,13.64,32.2,8.9,50304.11,7.69,91.23,32.96,NaN,ASU,KA32,2013-10-30,15.56,40.94,189.88,3.05,177.74,0.00
OCT172013,43.07,13.09,8.18,OCT172013,KA32,KA32_10042013_CRE,KA32_10172013_SPW,17.73,2013-11-04,replete-8.5,17.729167,1.0,922.50,0.453333,418.200000,22.86,0.72,8.56,NaN,10.8,12.97,4.39,0.02,141.42,1174.31,4.0,0.50,2.75,0.45,1.27,10.11,2.05,1.01,0.008,NaN,0.008,ASU,P1,KA32,8.38,16.20,28.3,9.3,30871.30,7.06,81.92,20.18,NaN,ASU,KA32,2013-11-04,18.92,37.88,176.55,6.42,194.43,0.18
OCT172013,43.35,11.17,6.44,OCT172013,KA32,KA32_10042013_CRE,KA32_10172013_SPW,6.73,2013-10-24,NaN,NaN,NaN,NaN,NaN,NaN,24.13,0.96,8.63,NaN,12.3,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,0.62,0.12,NaN,NaN,NaN,ASU,P2,KA32,8.49,NaN,NaN,NaN,47942.37,7.93,94.29,31.21,NaN,ASU,KA32,2013-10-24,22.79,24.90,186.99,4.56,198.29,0.00
OCT172013,43.40,NaN,7.65,OCT172013,KA32,KA32_10042013_CRE,KA32_10172013_SPW,12.74,2013-10-30,NaN,NaN,NaN,NaN,NaN,NaN,24.13,0.46,8.74,NaN,9.3,22.40,1.18,0.48,0.44,103.44,6.0,0.35,0.54,0.34,0.57,4.70,1.47,2.14,0.147,NaN,0.003,ASU,P2,KA32,8.45,NaN,NaN,NaN,39332.44,7.05,79.36,25.24,NaN,ASU,KA32,2013-10-30,15.56,40.94,189.88,3.05,177.74,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JUN172015,NaN,NaN,NaN,JUN172015,C046,C046_20150619_GTPooled,C046_20150619_GT,9.90,2015-06-29,"3x, 0.214",9.895833,1.0,512.50,0.436667,223.791667,23.70,0.43,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.69,3377.0,0.44,3.45,0.44,2.64,1.48,0.34,2.32,0.027,NaN,0.001,GT,P2,C046,7.96,25.18,31.1,19.1,53836.39,8.96,134.90,35.54,NaN,GT,C046,2015-06-29,24.32,56.66,304.52,1.62,250.85,0.00
JUN172015,NaN,NaN,NaN,JUN172015,C046,C046_20150619_GTPooled,C046_20150619_GT,9.90,2015-06-29,"3x, 0.214",9.895833,1.0,512.50,0.422222,216.388889,23.70,0.43,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.69,3378.0,0.43,2.75,0.42,1.99,0.77,0.31,1.50,0.021,NaN,0.001,GT,P3,C046,7.96,25.35,31.2,19.1,54684.10,7.35,110.93,36.19,NaN,GT,C046,2015-06-29,24.32,56.66,304.52,1.62,250.85,0.00
JUN172015,NaN,NaN,NaN,JUN172015,C046,C046_20150619_GTPooled,C046_20150619_GT,9.90,2015-06-29,"3x, 0.11",9.895833,1.0,256.25,0.387778,99.368056,23.70,0.43,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.69,3379.0,0.39,4.44,0.39,4.06,0.56,0.29,1.20,0.031,NaN,0.001,GT,P4,C046,7.94,25.09,30.9,19.1,55210.86,8.34,125.72,36.58,NaN,GT,C046,2015-06-29,24.32,56.66,304.52,1.62,250.85,0.00
